In [7]:
import numpy as np
import math
import dataclasses
from typing import Dict, Tuple
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from matplotlib import cm, colors
from numpy.polynomial.legendre import Legendre

# 1. LGL nodes, weights, and differentiation matrix

In [8]:
# =========================================================
# 1. LGL nodes, weights, and differentiation matrix
# =========================================================

def _lg_nodes_weights(a, b, N):
    xi, wi = np.polynomial.legendre.leggauss(N)  # on [-1,1]
    c = 0.5 * (b - a)
    m = 0.5 * (b + a)
    nodes = c * xi + m
    weights = c * wi
    return nodes, weights

def _lgl_nodes_weights(a, b, N):
    """
    Legendre-Gauss-Lobatto nodes/weights on [a,b].
    Using P_{N-1}(x) and its derivative for interior nodes.
    """
    if N < 2:
        raise ValueError("LGL requires N >= 2")

    # Legendre polynomial of degree N-1
    P = Legendre.basis(N - 1)   # P_{N-1}(x) on [-1,1]
    dP = P.deriv()

    # Interior roots of dP (if N>2)
    x_int = np.sort(dP.roots()) if N > 2 else np.array([], dtype=float)

    # Concatenate endpoints +-1
    x = np.concatenate(([-1.0], x_int, [1.0]))

    # Weights on [-1,1]
    PN1_vals = P(x)              # P_{N-1}(x_i)
    w = 2.0 / (N * (N - 1) * (PN1_vals ** 2))

    # Map to [a,b]
    c = 0.5 * (b - a)
    m = 0.5 * (b + a)
    nodes = c * x + m
    weights = c * w
    return nodes, weights


def lgl_diff_matrix(N: int) -> np.ndarray:
    """
    Construct the 1D LGL differentiation matrix D on given N.
    """
    Pn = Legendre.basis(N-1)
    x, _ = _lgl_nodes_weights(-1.0, 1.0, N)
    
    # 評估 P_N(x) 在標準格點上的值
    Pn_x = Pn(x)
    
    # 初始化導數矩陣
    D = np.zeros((N, N))
    
    # 1. 先計算非對角線元素
    for i in range(N):
        for j in range(N):
            if i != j:
                # 公式: (P_N(xi) / P_N(xj)) * (1 / (xi - xj))
                denominator = (x[i] - x[j]) * Pn_x[j]
                if np.abs(denominator) < 1e-15:
                     # 理論上 LGL 節點不重複，不會發生除以零，除非精度問題
                    D[i, j] = 0.0 
                else:
                    D[i, j] = Pn_x[i] / denominator
    
    # 2. 使用 Negative Sum Trick 計算對角線元素
    # 這比直接使用公式 D_ii = -xi / (2*(1-xi^2)) 更穩健，且自動滿足 D * const = 0
    for i in range(N):
        D[i, i] = -np.sum(D[i, :])
        
    return D

# 2. Topology & Geometry

In [9]:
class CubedSphereTopology:
    """
    處理 Cubed-Sphere 面與面之間的連接關係
    """
    def __init__(self):
        # 映射表: FaceName -> Index
        self.FACE_MAP = ["P1", "P2", "P3", "P4", "P5", "P6"]
        self.FACE_IDX = {name: i for i, name in enumerate(self.FACE_MAP)}
        
        # 連接表: (FaceIdx, Side) -> (NeighborFaceIdx, NeighborSide, SwapXY, Reverse)
        # Sides: 0:West(Alpha=-1), 1:East(Alpha=1), 2:South(Beta=-1), 3:North(Beta=1)
        self.CONN_TABLE = {
            (0, 0): (3, 1, False, False), (0, 1): (1, 0, False, False), (0, 2): (5, 3, False, False), (0, 3): (4, 2, False, False),
            (1, 0): (0, 1, False, False), (1, 1): (2, 0, False, False), (1, 2): (5, 1, True, True),   (1, 3): (4, 1, True, False),
            (2, 0): (1, 1, False, False), (2, 1): (3, 0, False, False), (2, 2): (5, 2, False, True),  (2, 3): (4, 3, False, True),
            (3, 0): (2, 1, False, False), (3, 1): (0, 0, False, False), (3, 2): (5, 0, True, False),  (3, 3): (4, 0, True, True),
            (4, 0): (3, 3, True, True),   (4, 1): (1, 3, True, False),  (4, 2): (0, 3, False, False), (4, 3): (2, 3, False, True),
            (5, 0): (3, 2, True, False),  (5, 1): (1, 2, True, True),   (5, 2): (2, 2, False, True),  (5, 3): (0, 2, False, False),
        }

    def get_neighbor_data(self, global_state, face_idx, side_idx):
        """
        從全域狀態中提取鄰居邊界的數據，並處理翻轉與旋轉
        """
        nbr_face, nbr_side, swap, reverse = self.CONN_TABLE[(face_idx, side_idx)]
        nbr_data = global_state[nbr_face]
        
        # 提取切片
        if nbr_side == 0: slice_data = nbr_data[0, :]   # West edge
        elif nbr_side == 1: slice_data = nbr_data[-1, :]  # East edge
        elif nbr_side == 2: slice_data = nbr_data[:, 0]   # South edge
        elif nbr_side == 3: slice_data = nbr_data[:, -1]  # North edge
        
        if reverse:
            slice_data = slice_data[::-1]
            
        return slice_data

@dataclasses.dataclass
class FaceGrid:
    """單一個面的網格數據與預計算的物理量"""
    name: str
    alpha: np.ndarray
    beta:  np.ndarray
    walpha: np.ndarray # Weights
    wbeta: np.ndarray
    x: np.ndarray
    y: np.ndarray
    X: np.ndarray      # 3D Cart X
    Y: np.ndarray      # 3D Cart Y
    Z: np.ndarray      # 3D Cart Z
    
    # --- Pre-computed Metric Terms ---
    sqrt_g: np.ndarray = None  # Jacobian
    u1: np.ndarray = None      # Contravariant velocity 1 (static wind)
    u2: np.ndarray = None      # Contravariant velocity 2 (static wind)
    div_u: np.ndarray = None   # Divergence of static wind field
    
    # --- Dynamic State ---
    phi: np.ndarray = None     # Scalar field

class CubedSphereEquiangular:
    """幾何轉換與 A 矩陣計算"""
    def __init__(self, R: float = 1.0):
        self.R = float(R)
        self.a = self.R / math.sqrt(3.0)

    def generate_face(self, N: int, face: str) -> FaceGrid:
        # LGL Nodes [-pi/4, pi/4]
        nodes, weights = _lgl_nodes_weights(-np.pi/4, np.pi/4, N)
        AA, BB = np.meshgrid(nodes, nodes, indexing="ij")
        
        # Equiangular coordinates to local x, y
        x = self.a * np.tan(AA)
        y = self.a * np.tan(BB)
        
        # Local x, y to Global X, Y, Z
        X, Y, Z = self._xyz_from_xy(x, y, face)
        
        fg = FaceGrid(name=face, alpha=AA, beta=BB, walpha=weights, wbeta=weights,
                      x=x, y=y, X=X, Y=Y, Z=Z)
        
        # Calculate sqrt_g (Jacobian) immediately
        rho = np.sqrt(1.0 + np.tan(AA)**2 + np.tan(BB)**2)
        fg.sqrt_g = (self.R**2) / (rho**3 * (np.cos(AA)**2) * (np.cos(BB)**2))
        
        return fg

    def _xyz_from_xy(self, x, y, face):
        r = np.sqrt(self.a**2 + x**2 + y**2)
        s = self.R / r
        if face == "P1":   return s*self.a, s*x, s*y
        elif face == "P2": return s*(-x), s*self.a, s*y
        elif face == "P3": return s*(-self.a), s*(-x), s*y
        elif face == "P4": return s*x, s*(-self.a), s*y
        elif face == "P5": return s*(-y), s*x, s*self.a
        elif face == "P6": return s*y, s*x, s*(-self.a)
        else: raise ValueError(f"Unknown face: {face}")

    @staticmethod
    def lonlat_from_xyz(X, Y, Z):
        lam = np.arctan2(Y, X)
        theta = np.arcsin(Z / np.sqrt(X**2 + Y**2 + Z**2))
        return lam, theta

    def solid_body_wind(self, X, Y, Z, alpha0, u0):
        """計算剛體旋轉風場 (u, v) in Spherical components"""
        lam, theta = self.lonlat_from_xyz(X, Y, Z)
        u_sphere = u0 * (np.cos(alpha0) * np.cos(theta) + np.sin(alpha0) * np.cos(lam) * np.sin(theta))
        v_sphere = -u0 * np.sin(alpha0) * np.sin(lam)
        return u_sphere, v_sphere

    def compute_contravariant_vel(self, fg: FaceGrid, u_sphere, v_sphere):
        """計算逆變速度 u1, u2"""
        lam, theta = self.lonlat_from_xyz(fg.X, fg.Y, fg.Z)
        
        # A matrix components construction
        sec2_a = 1.0 / (np.cos(fg.alpha) ** 2)
        sec2_b = 1.0 / (np.cos(fg.beta) ** 2)
        c_lam, s_lam = np.cos(lam), np.sin(lam)
        c_th, s_th = np.cos(theta), np.sin(theta)
        
        A = np.zeros(fg.alpha.shape + (2, 2))
        
        # Populate A matrix based on face (Using simplified mapping logic)
        # Note: This part follows the standard Sadourny/Nair formalism
        if fg.name == "P1":
            A[...,0,0], A[...,0,1] = self.R * c_lam**2 * c_th * sec2_a, 0.0
            A[...,1,0], A[...,1,1] = -self.R * s_lam * c_lam * s_th * c_th * sec2_a, self.R * c_lam * c_th**2 * sec2_b
        elif fg.name == "P2":
            A[...,0,0], A[...,0,1] = self.R * s_lam**2 * c_th * sec2_a, 0.0
            A[...,1,0], A[...,1,1] = self.R * s_lam * c_lam * s_th * c_th * sec2_a, self.R * s_lam * c_th**2 * sec2_b
        elif fg.name == "P3":
            A[...,0,0], A[...,0,1] = self.R * c_lam**2 * c_th * sec2_a, 0.0
            A[...,1,0], A[...,1,1] = -self.R * s_lam * c_lam * s_th * c_th * sec2_a, -self.R * c_lam * c_th**2 * sec2_b
        elif fg.name == "P4":
            A[...,0,0], A[...,0,1] = self.R * s_lam**2 * c_th * sec2_a, 0.0
            A[...,1,0], A[...,1,1] = self.R * s_lam * c_lam * s_th * c_th * sec2_a, -self.R * s_lam * c_th**2 * sec2_b
        elif fg.name == "P5": # Top
            A[...,0,0], A[...,0,1] = self.R * c_lam * s_th * sec2_a, self.R * s_lam * s_th * sec2_b
            A[...,1,0], A[...,1,1] = -self.R * s_lam * s_th**2 * sec2_a, self.R * c_lam * s_th**2 * sec2_b
        elif fg.name == "P6": # Bottom
            A[...,0,0], A[...,0,1] = -self.R * c_lam * s_th * sec2_a, self.R * s_lam * s_th * sec2_b
            A[...,1,0], A[...,1,1] = self.R * s_lam * s_th**2 * sec2_a, self.R * c_lam * s_th**2 * sec2_b

        # Inversion A^-1 * [u, v]^T
        det = A[...,0,0]*A[...,1,1] - A[...,0,1]*A[...,1,0]
        inv_det = 1.0 / det
        
        u1 = inv_det * (A[...,1,1]*u_sphere - A[...,0,1]*v_sphere)
        u2 = inv_det * (-A[...,1,0]*u_sphere + A[...,0,0]*v_sphere)
        return u1, u2

# 3. Solver Core

In [10]:
class CubedSphereAdvectionSolver:
    def __init__(self, N, R=1.0, u0=2*np.pi, alpha0=0.0):
        self.N = N
        self.R = R
        self.u0 = u0
        self.topology = CubedSphereTopology()
        self.geometry = CubedSphereEquiangular(R)
        self.D = lgl_diff_matrix(N)
        
        # 初始化 Grid Faces
        self.faces = {}
        for fname in self.topology.FACE_MAP:
            self.faces[fname] = self.geometry.generate_face(N, fname)
            
        # 預計算靜態風場的逆變速度與散度
        self._precompute_static_fields(alpha0, u0)
        
        # LSRK5 Coefficients
        self.rk_a = np.array([0.0, 
                         -567301805773.0/1357537059087.0, 
                         -2404267990393.0/2016746695238.0, 
                         -3550918686646.0/2091501179385.0, 
                         -1275806237668.0/842570457699.0])
        self.rk_b = np.array([1432997174477.0/9575080441755.0, 
                         5161836677717.0/13612068292357.0, 
                         1720146321549.0/2090206949498.0, 
                         3134564353537.0/4481467310338.0, 
                         2277821191437.0/14882151754819.0])

    def _precompute_static_fields(self, alpha0, u0):
        """
        因為是 Advection of Passive Scalar，風場固定。
        預先計算所有面的 u1, u2 以及 div(u)。
        """
        for fname, fg in self.faces.items():
            # 1. 物理風場 (Spherical)
            u_sph, v_sph = self.geometry.solid_body_wind(fg.X, fg.Y, fg.Z, alpha0, u0)
            
            # 2. 逆變風場 (Contravariant)
            u1, u2 = self.geometry.compute_contravariant_vel(fg, u_sph, v_sph)
            fg.u1 = u1
            fg.u2 = u2
            
            # 3. 預計算 Divergence of Velocity (Correction Term)
            # div(u) = (1/sqrt_g) * [ d/da(sqrt_g * u1) + d/db(sqrt_g * u2) ]
            term1 = self.D @ (fg.sqrt_g * u1)
            term2 = (fg.sqrt_g * u2) @ self.D.T
            fg.div_u = (1.0 / fg.sqrt_g) * (term1 + term2)

    def set_initial_condition(self, type="gaussian", h0=1.0, r0=None):
        """設定初始純量場 phi"""
        if r0 is None: r0 = self.R / 3.0
        
        center_lon = 0
        center_lat = 0.0
        
        # 轉換為 (6, N, N) 的狀態陣列
        state = np.zeros((6, self.N, self.N))
        
        for i, fname in enumerate(self.topology.FACE_MAP):
            fg = self.faces[fname]
            lam, theta = self.geometry.lonlat_from_xyz(fg.X, fg.Y, fg.Z)
            
            # Great Circle Distance
            sin_c, cos_c = np.sin(center_lat), np.cos(center_lat)
            sin_th, cos_th = np.sin(theta), np.cos(theta)
            
            cos_sigma = sin_th * sin_c + cos_th * cos_c * np.cos(lam - center_lon)
            r_d = self.R * np.arccos(np.clip(cos_sigma, -1.0, 1.0))
            
            if type == "gaussian":
                fg.phi = h0 * np.exp(-(r_d / r0)**2)
            elif type == "cosine":
                fg.phi = np.where(r_d < r0, 0.5 * h0 * (1.0 + np.cos(np.pi * r_d / r0)), 0.0)
            
            state[i] = fg.phi
            
        return state

    def compute_rhs(self, global_phi):
        """
        計算 d(phi)/dt 的右式 (RHS)
        RHS = - (Skew-Symmetric Divergence) + (SAT Penalty)
        """
        rhs = np.zeros_like(global_phi)
        map_factor = 4.0 / np.pi
        
        # 微分輔助函數
        def da(F): return self.D @ F * map_factor
        def db(F): return F @ self.D.T * map_factor
        
        for i, fname in enumerate(self.topology.FACE_MAP):
            fg = self.faces[fname]
            phi = global_phi[i]
            
            # --- 1. Volume Integral (Skew-Symmetric Form) ---
            # Flux Term: (1/J) * div(J * u * phi)
            flux = (1.0 / fg.sqrt_g) * (da(fg.sqrt_g * fg.u1 * phi) + db(fg.sqrt_g * fg.u2 * phi))
            
            # Advective Term: u * grad(phi)
            advect = fg.u1 * da(phi) + fg.u2 * db(phi)
            
            # Correction Term: phi * div(u) (Already precomputed)
            correction = phi * fg.div_u
            
            # Skew-Symmetric Average
            skew_div = 0.5 * (flux + advect + correction)
            
            # --- 2. Surface Integral (SAT Penalty) ---
            penalty = np.zeros_like(phi)
            
            # 輔助函數: 計算單邊懲罰
            def sat(vn, q_in, q_out, w_metric):
                # 只有流進來 (Inflow, Vn < 0) 時才懲罰
                flux_diff = 0.5 * (vn - np.abs(vn)) * (q_in - q_out)
                return flux_diff / w_metric

            # West (Alpha=-1) -> Vn = -u1
            q_out = self.topology.get_neighbor_data(global_phi, i, 0)
            penalty[0, :] += sat(-fg.u1[0, :], phi[0, :], q_out, fg.sqrt_g[0, :] * fg.walpha[0])
            
            # East (Alpha=1) -> Vn = u1
            q_out = self.topology.get_neighbor_data(global_phi, i, 1)
            penalty[-1, :] += sat(fg.u1[-1, :], phi[-1, :], q_out, fg.sqrt_g[-1, :] * fg.walpha[-1])
            
            # South (Beta=-1) -> Vn = -u2
            q_out = self.topology.get_neighbor_data(global_phi, i, 2)
            penalty[:, 0] += sat(-fg.u2[:, 0], phi[:, 0], q_out, fg.sqrt_g[:, 0] * fg.wbeta[0])
            
            # North (Beta=1) -> Vn = u2
            q_out = self.topology.get_neighbor_data(global_phi, i, 3)
            penalty[:, -1] += sat(fg.u2[:, -1], phi[:, -1], q_out, fg.sqrt_g[:, -1] * fg.wbeta[-1])
            
            # Final Equation: dphi/dt = - Div + Penalty
            rhs[i] = -skew_div + penalty
            
        return rhs

    def run_simulation(self, T_final, CFL=1.0):
        """執行時間積分迴圈"""
        # 估計 dt
        dt = (CFL / (2 * self.u0)) * (2 / self.N**2)
        
        state = self.set_initial_condition(type="gaussian", h0=1.0)
        du = np.zeros_like(state) # LSRK residual accumulator
        
        t = 0.0
        step = 0
        
        print(f"=== Starting Simulation ===")
        print(f"N={self.N}, CFL={CFL}, dt={dt:.5f}, T_final={T_final}")
        
        while t < T_final:
            if t + dt > T_final:
                dt = T_final - t
            
            # LSRK5 Loop
            local_state = state.copy()
            for k in range(5):
                rhs = self.compute_rhs(local_state)
                du = self.rk_a[k] * du + dt * rhs
                local_state = local_state + self.rk_b[k] * du
            
            state = local_state
            t += dt
            step += 1
            
            if step % 50 == 0:
                print(f"Step {step}: t={t:.4f}, Max={np.max(state):.6f}, Min={np.min(state):.6f}")
                
        print("=== Simulation Complete ===")
        return state

# 4. Main

In [11]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

def plot_results(solver, final_state):
    """
    繪製 3D 球面結果，採用綠-黃-紅配色 (Green -> Yellow -> Red)
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection="3d")
    
    # 1. 定義綠到底，紅峰值的 Colormap
    colors_list = [
        (0.0, "green"),
        (0.5, "yellow"),
        (1.0, "red"),
    ]
    cmap = LinearSegmentedColormap.from_list("green_red", colors_list)
    
    # 2. 設定 Normalization (依據 final_state 的最大最小值)
    # 如果希望像第二段程式碼一樣固定從 0 開始，可將 vmin 改為 0.0
    vmin, vmax = np.min(final_state), np.max(final_state)
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    
    # 3. 繪製各面
    for i, fname in enumerate(solver.topology.FACE_MAP):
        fg = solver.faces[fname]
        data = final_state[i]
        
        # 將數據轉換為對應的顏色值 (RGBA)
        rgba = cmap(norm(data))
        
        # 繪製曲面 (加入 alpha=0.6 透明度)
        ax.plot_surface(fg.X, fg.Y, fg.Z, rstride=1, cstride=1,
                       facecolors=rgba,
                       linewidth=0, antialiased=False, shade=False, 
                       alpha=0.6)
    
    # 4. Colorbar 設定
    mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
    mappable.set_array(final_state)
    
    # shrink 調整 colorbar 大小，pad 調整與圖的距離
    cbar = plt.colorbar(mappable, ax=ax, shrink=0.8, pad=0.05)
    cbar.set_label("Phi (m)")
    
    # 5. 圖表裝飾與視角
    ax.set_title("Advection on Cubed-Sphere (Green-Red Style)")
    ax.set_box_aspect([1, 1, 1])
    
    # 設定視角 (Elevation 和 Azimuth)
    # ax.view_init(elev=elev, azim=azim)
    
    plt.tight_layout()
    plt.show()

# 5. exampe

## 5.2 Animation

In [12]:
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# from mpl_toolkits.mplot3d import Axes3D
# from matplotlib import cm, colors
# from matplotlib.colors import LinearSegmentedColormap
# from IPython.display import HTML

# # =========================================================
# # 1. 修改版的模擬函式 (紀錄歷史狀態) - [未更動]
# # =========================================================
# def run_simulation_with_history(solver, T_final, dt_output, CFL=0.5):
#     """
#     執行模擬並依據 dt_output 紀錄狀態
#     """
#     # 估計數值積分用的 dt (LSRK dt)
#     dt_solver = (CFL / (2 * solver.u0)) * (2 / solver.N**2)
    
#     if dt_solver > dt_output:
#         dt_solver = dt_output

#     state = solver.set_initial_condition(type="gaussian", h0=1.0)
#     du = np.zeros_like(state)
    
#     t = 0.0
#     next_output_time = 0.0
    
#     history = []
#     times = []
    
#     history.append(state.copy())
#     times.append(t)
#     next_output_time += dt_output
    
#     print(f"=== Starting Animation Simulation ===")
#     print(f"Solver dt={dt_solver:.6f}, Output dt={dt_output:.6f}")
    
#     step = 0
#     while t < T_final:
#         current_dt = dt_solver
#         if t + current_dt > T_final:
#             current_dt = T_final - t
        
#         local_state = state.copy()
#         for k in range(5):
#             rhs = solver.compute_rhs(local_state)
#             du = solver.rk_a[k] * du + current_dt * rhs
#             local_state = local_state + solver.rk_b[k] * du
        
#         state = local_state
#         t += current_dt
#         step += 1
        
#         if t >= next_output_time - 1e-9:
#             history.append(state.copy())
#             times.append(t)
#             next_output_time += dt_output
            
#     print(f"=== Simulation Complete. Total frames: {len(history)} ===")
#     return history, times

# # =========================================================
# # 2. [修改後] 製作動畫的函式：採用格點平均繪製
# # =========================================================
# def create_animation(solver, history, times, interval=50, save_gif=True, filename="advection_avg.gif", elev=20, azim=-35):
#     """
#     建立 3D 動畫 (綠-黃-紅 配色風格) - 採用格點平均 (Cell Averaged) 繪製
#     """
#     fig = plt.figure(figsize=(10, 8), dpi=200)
#     ax = fig.add_subplot(111, projection="3d")
    
#     # 1. 定義綠到底，紅峰值的 Colormap
#     colors_list = [
#         (0.0, "green"),
#         (0.5, "yellow"),
#         (1.0, "red"),
#     ]
#     cmap = LinearSegmentedColormap.from_list("green_red", colors_list)
    
#     # 2. 計算全域最大最小值
#     all_min = min(np.min(frame) for frame in history)
#     all_max = max(np.max(frame) for frame in history)
#     norm = colors.Normalize(vmin=all_min, vmax=all_max)
    
#     # Colorbar
#     mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
#     mappable.set_array([])
#     fig.colorbar(mappable, ax=ax, shrink=0.7, label="Phi (Cell Average)", pad=0.05)
    
#     def update(frame_idx):
#         ax.clear()
        
#         current_state = history[frame_idx]
#         current_time = times[frame_idx]
        
#         # 繪製六個面
#         for i, fname in enumerate(solver.topology.FACE_MAP):
#             fg = solver.faces[fname]
#             data = current_state[i] # data shape is (N+1, N+1)
            
#             # --- [核心修改] 計算格點平均值 ---
#             # 取四個角落的平均值作為該 cell 中心的代表值
#             # data 的維度是 (N+1, N+1)，平均後的 data_avg 維度會是 (N, N)
#             # 這正好對應 plot_surface 生成的 N*N 個面 (faces)
#             data_avg = 0.25 * (data[:-1, :-1] + data[1:, :-1] + 
#                                data[:-1, 1:] + data[1:, 1:])
            
#             # 將平均值轉為顏色 (N, N, 4)
#             face_colors = cmap(norm(data_avg))
            
#             # 繪製曲面
#             # 關鍵參數：
#             # 1. facecolors=face_colors: 指定每一個面的顏色
#             # 2. shade=False: 關閉光影渲染，確保顏色呈現我們指定的平均色，造成「平坦」的視覺效果
#             ax.plot_surface(fg.X, fg.Y, fg.Z, rstride=1, cstride=1,
#                            facecolors=face_colors,
#                            linewidth=0, antialiased=False, shade=False,
#                            alpha=0.8) # 稍微提高不透明度讓格點更明顯
        
#         ax.set_title(f"Cubed-Sphere Advection (Cell Average)\nTime: {current_time:.3f}")
#         ax.set_box_aspect([1, 1, 1])
#         ax.set_axis_off() 
#         ax.view_init(elev=elev, azim=azim) # 若需要固定視角可取消註解
        
#         return ax,

#     ani = animation.FuncAnimation(fig, update, frames=len(history), interval=interval, blit=False)
    
#     if save_gif:
#         print(f"Saving animation to {filename}...")
#         try:
#             ani.save(filename, writer='pillow', fps=1000/interval)
#             print("Done.")
#         except Exception as e:
#             print(f"Could not save GIF: {e}")

#     plt.close(fig)
#     return ani

# # =========================================================
# # 3. 執行範例
# # =========================================================

# # 初始化 Solver (使用 N=32 讓格點顆粒感較明顯，方便觀察平均效果)
# N_anim = 32 
# solver_anim = CubedSphereAdvectionSolver(N=N_anim, R=1.0, u0=2*np.pi, alpha0=-np.pi/4)

# T_total = 2.0        
# dt_frame = 0.05      # 稍微加大間隔減少張數

# # 1. 執行模擬
# history, times = run_simulation_with_history(solver_anim, T_total, dt_frame, CFL=1)

# # 2. 產生動畫 (會使用新的平均繪製法)
# ani = create_animation(solver_anim, history, times, interval=100, save_gif=True, filename="cubed_sphere_avg.gif")

# # 3. 顯示
# HTML(ani.to_jshtml())

In [13]:
# import pyvista as pv
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib import animation
# from IPython.display import HTML

# # =========================================================
# # 【修正 1】增加動畫檔案大小上限 (例如設為 200 MB)
# # 這樣即使 DPI=300 也不會因為超過 20MB 而報錯
# # =========================================================
# plt.rcParams['animation.embed_limit'] = 200.0 

# # ---------------------------------------------------------
# # 1. 輔助函數
# # ---------------------------------------------------------
# def build_cubed_sphere_mesh(solver):
#     faces = []
#     for fname in solver.topology.FACE_MAP:
#         fg = solver.faces[fname]
#         nx, ny = fg.X.shape
#         points = np.column_stack([
#             fg.X.ravel(), fg.Y.ravel(), fg.Z.ravel()
#         ])
#         grid = pv.StructuredGrid()
#         grid.points = points
#         grid.dimensions = (nx, ny, 1)
#         faces.append(grid)
#     return faces

# # ---------------------------------------------------------
# # 2. 核心：高解析度動畫函數
# # ---------------------------------------------------------
# def create_high_res_animation(solver, history, times, threshold=0.1, dpi=150):
    
#     # --- 計算所需的像素大小 ---
#     pixel_size = int(6 * dpi)
    
#     # --- A. 設定 PyVista ---
#     pv.set_plot_theme("dark")
#     plotter = pv.Plotter(off_screen=True, window_size=(pixel_size, pixel_size))
#     plotter.set_background("black")
    
#     face_meshes = build_cubed_sphere_mesh(solver)

#     for mesh in face_meshes:
#         # 為了遮擋背面，建議 opacity 設為 1.0；若您原本想透視可維持 0.5
#         plotter.add_mesh(mesh, color="black", opacity=0.5, smooth_shading=False)

#         edges = mesh.extract_all_edges()
#         edges.points *= 1.002 
#         plotter.add_mesh(edges, color="white", line_width=1.0, opacity=1.0)

#     X_all, Y_all, Z_all = [], [], []
#     for fname in solver.topology.FACE_MAP:
#         fg = solver.faces[fname]
#         X_all.append(fg.X.ravel())
#         Y_all.append(fg.Y.ravel())
#         Z_all.append(fg.Z.ravel())
#     X_all = np.concatenate(X_all)
#     Y_all = np.concatenate(Y_all)
#     Z_all = np.concatenate(Z_all)
    
#     current_actor = [None] 

#     plotter.camera_position = 'iso'
#     plotter.camera.zoom(1.2)

#     # --- B. Matplotlib 初始化 ---
#     fig, ax = plt.subplots(figsize=(6, 6), dpi=dpi)
#     ax.axis('off')
#     fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    
#     plotter.render()
#     initial_img = plotter.screenshot(return_img=True)
#     im = ax.imshow(initial_img, animated=True)

#     # --- C. 更新函數 ---
#     def update(frame_idx):
#         state = history[frame_idx]
#         t = times[frame_idx]
        
#         phi_all = np.concatenate([face.flatten() for face in state])
#         mask = phi_all > threshold

#         if current_actor[0] is not None:
#             plotter.remove_actor(current_actor[0])

#         pts = np.column_stack([X_all[mask], Y_all[mask], Z_all[mask]])
        
#         if pts.shape[0] > 0:
#             pts_offset = pts * 1.01
            
#             new_particles = pv.PolyData(pts_offset)
#             new_particles["phi"] = phi_all[mask]
            
#             current_actor[0] = plotter.add_mesh(
#                 new_particles,
#                 scalars="phi",
#                 cmap="inferno",
#                 # 動態調整粒子大小，確保高 DPI 下不會變成微塵
#                 point_size=6 * (dpi / 100), 
#                 render_points_as_spheres=True,
#                 opacity=1.0,
#                 reset_camera=False,
#                 show_scalar_bar=False
#             )
#         else:
#             current_actor[0] = None
        
#         plotter.add_text(
#             f"Particle Advection\nTime = {t:.3f}",
#             name="time_label",
#             color="white",
#             font_size=14, 
#             position='upper_left'
#         )

#         if frame_idx == 0:
#              plotter.add_scalar_bar("Phi", color="white")

#         plotter.render()
#         image = plotter.screenshot(return_img=True)
#         im.set_array(image)
#         return [im]

#     ani = animation.FuncAnimation(
#         fig, update, frames=len(history), interval=50, blit=True
#     )
    
#     plt.close(fig) # 關閉多餘的靜態圖
    
#     return ani

# # =========================================================
# # 執行範例
# # =========================================================

# # 初始化 Solver (假設您的 class 已經定義好)
# N_anim = 32
# solver_anim = CubedSphereAdvectionSolver(
#     N=N_anim,
#     R=1.0,
#     u0=2*np.pi,
#     alpha0=-np.pi/4
# )

# T_total = 1.0
# dt_frame = 0.02

# history, times = run_simulation_with_history(
#     solver_anim,
#     T_total,
#     dt_frame,
#     CFL=1
# )

# # 產生 Animation
# # 【建議】雖然調高了上限，但建議 dpi 設為 150 即可，300 會導致網頁載入非常慢
# ani = create_high_res_animation(
#     solver_anim,
#     history,
#     times,
#     threshold=0.1,
#     dpi=300  # 建議改為 150；若堅持要 300，因為上方已調高 embed_limit，現在也可以跑了
# )

# HTML(ani.to_jshtml())

In [14]:
# # 方法：將 jshtml 的字串寫入檔案
# html_content = ani.to_jshtml()

# with open("particle_advection.html", "w") as f:
#     f.write(html_content)

# print("HTML 檔案已儲存：particle_advection.html")

# # writer='pillow' 是最通用的內建寫入器
# # fps: 控制播放速度
# # dpi: 確保輸出的 GIF 清晰度與您預覽的一致
# ani.save("particle_advection.gif", writer="pillow", fps=30, dpi=150)

# print("GIF 檔案已儲存：particle_advection.gif")